In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy, pandas, os, matplotlib.pyplot as rvmp, seaborn as rvse, datetime, random
from sklearn import metrics, decomposition, model_selection, utils, pipeline,preprocessing, feature_selection, tree
from plotly import express
import plotly.graph_objects as go
import pickle
from IPython.display import display, HTML
from sklearn import tree,linear_model,svm,neighbors,neural_network,ensemble

##  **DATA READING**

In [3]:
SIIData=pandas.read_csv("Startups_India_Investments.csv")
if "Unnamed: 0" in SIIData.columns:
    SIIData=SIIData.drop("Unnamed: 0",axis=1)
SIIData.head()

,Startup_ID,Name,Sector,Sub-Sector,City,State,Founded_Year,Founder_Name,Funding_Stage,Investment_Type,...,Exit_Status,Acquisition_Details,Funding_Date,Pitch_Deck_Link,Social_Media_Followers,Competitors,Patents,ESG_Score,Diversity_Index,Net_Impact_Score
0,a713cd4f,Startup_0,Retail,Sub_2,Pune,Maharashtra,2022,Founder_356,Series D,Grant,...,No,NaN,2023-07-08,http://example.com/pitchdeck/0,27648,Competitor_88,11,5.32,39.33,4.71
1,b75f6987,Startup_1,EdTech,Sub_2,Pune,Karnataka,2019,Founder_93,IPO,Convertible Note,...,Yes,Acquirer_185,2017-01-02,http://example.com/pitchdeck/1,89143,Competitor_66,17,2.50,43.19,5.03
2,d5ba7ec2,Startup_2,AgriTech,Sub_7,Chennai,Tamil Nadu,2010,Founder_951,Series A,Grant,...,Yes,NaN,2022-06-20,http://example.com/pitchdeck/2,62445,Competitor_93,4,5.13,36.11,5.71
3,5190a643,Startup_3,HealthTech,Sub_8,Mumbai,Telangana,2007,Founder_538,IPO,Debt,...,Yes,NaN,2021-01-11,http://example.com/pitchdeck/3,30184,Competitor_31,13,9.14,31.38,9.29
4,a6801a40,Startup_4,HealthTech,Sub_6,Bangalore,Maharashtra,2019,Founder_69,Seed,Grant,...,No,NaN,2015-07-22,http://example.com/pitchdeck/4,56494,Competitor_27,0,8.10,78.20,1.12


In [4]:
Msngs=SIIData.isna().sum()
Msngs

Startup_ID                    0
Name                          0
Sector                        0
Sub-Sector                    0
City                          0
State                         0
Founded_Year                  0
Founder_Name                  0
Funding_Stage                 0
Investment_Type               0
Amount_Raised                 0
Investors_Count               0
Lead_Investor                 0
Co-Investors                  0
Valuation_Post_Funding        0
Revenue                       0
Profitability                 0
Number_of_Employees           0
Tech_Stack                    0
Primary_Product               0
Customer_Base_Size            0
Growth_Rate                   0
Exit_Status                   0
Acquisition_Details       10027
Funding_Date                  0
Pitch_Deck_Link               0
Social_Media_Followers        0
Competitors                   0
Patents                       0
ESG_Score                     0
Diversity_Index               0
Net_Impa

In [5]:
nmftsii=SIIData.dtypes[SIIData.dtypes!='object'].index.tolist()
obftsii=SIIData.dtypes[SIIData.dtypes=='object'].index.tolist()
for n in nmftsii:
    SIIData[n]=SIIData[n].fillna(SIIData[n].std())
for o in obftsii:
    SIIData[o]=SIIData[o].fillna(SIIData[o].mode()[0])
print(SIIData.isna().sum())
print(SIIData.info())

Startup_ID                0
Name                      0
Sector                    0
Sub-Sector                0
City                      0
State                     0
Founded_Year              0
Founder_Name              0
Funding_Stage             0
Investment_Type           0
Amount_Raised             0
Investors_Count           0
Lead_Investor             0
Co-Investors              0
Valuation_Post_Funding    0
Revenue                   0
Profitability             0
Number_of_Employees       0
Tech_Stack                0
Primary_Product           0
Customer_Base_Size        0
Growth_Rate               0
Exit_Status               0
Acquisition_Details       0
Funding_Date              0
Pitch_Deck_Link           0
Social_Media_Followers    0
Competitors               0
Patents                   0
ESG_Score                 0
Diversity_Index           0
Net_Impact_Score          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12428 entries, 0 to 12427
Data columns (t

In [6]:
SIIData.head()

,Startup_ID,Name,Sector,Sub-Sector,City,State,Founded_Year,Founder_Name,Funding_Stage,Investment_Type,...,Exit_Status,Acquisition_Details,Funding_Date,Pitch_Deck_Link,Social_Media_Followers,Competitors,Patents,ESG_Score,Diversity_Index,Net_Impact_Score
0,a713cd4f,Startup_0,Retail,Sub_2,Pune,Maharashtra,2022,Founder_356,Series D,Grant,...,No,Acquirer_429,2023-07-08,http://example.com/pitchdeck/0,27648,Competitor_88,11,5.32,39.33,4.71
1,b75f6987,Startup_1,EdTech,Sub_2,Pune,Karnataka,2019,Founder_93,IPO,Convertible Note,...,Yes,Acquirer_185,2017-01-02,http://example.com/pitchdeck/1,89143,Competitor_66,17,2.50,43.19,5.03
2,d5ba7ec2,Startup_2,AgriTech,Sub_7,Chennai,Tamil Nadu,2010,Founder_951,Series A,Grant,...,Yes,Acquirer_429,2022-06-20,http://example.com/pitchdeck/2,62445,Competitor_93,4,5.13,36.11,5.71
3,5190a643,Startup_3,HealthTech,Sub_8,Mumbai,Telangana,2007,Founder_538,IPO,Debt,...,Yes,Acquirer_429,2021-01-11,http://example.com/pitchdeck/3,30184,Competitor_31,13,9.14,31.38,9.29
4,a6801a40,Startup_4,HealthTech,Sub_6,Bangalore,Maharashtra,2019,Founder_69,Seed,Grant,...,No,Acquirer_429,2015-07-22,http://example.com/pitchdeck/4,56494,Competitor_27,0,8.10,78.20,1.12
